In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importerer nødvendige ressurser og setter opp aitextgen-modellen i Python. 

In [ ]:
from aitextgen import aitextgen
import matplotlib.pyplot as plt
from aitextgen.TokenDataset import TokenDataset


ai = aitextgen(to_gpu=True)

In [ ]:
ai.generate(max_length=100, prompt="All the mountains")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Never have I ever")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Something tells me")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Give meaning to")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Say it ain't so")

Produktet av modellen kan ikke forveksles med poesi, og vi kommer derfor ikke til å bry oss med å teste troverdigheten før trening. 

Nå forberederer vi datasett på poesi, før vi trener modellen.

In [ ]:
df = pd.read_csv('../input/modern-renaissance-poetry/all.csv')
list_mrp = list(df[df.content == df.content].content)

df1 = pd.read_csv('../input/complete-poetryfoundationorg-dataset/kaggle_poem_dataset.csv')
list_pfd = list(df1[df1.Content == df1.Content].Content)

full_list = list_mrp + list_pfd
training_data = TokenDataset(texts=full_list)

In [ ]:
ai.train(training_data, 
        line_by_line=True,
        from_cache=False,
        num_steps=1000,
        learning_rate=1e-5,
        batch_size=1
        )

In [ ]:
ai.generate(max_length=100, prompt="All the mountains")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Never have I ever")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Something tells me")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Give meaning to")
print("----------------------------------------------------")
ai.generate(max_length=100, prompt="Say it ain't so")

In [ ]:
rank1 = [2,2,4,2,1,6,4,2,1,4,6,4,4,2,1,2,3]
rank2 = [6,1,2,3,2,4,6,5,5,5,2,6,1,3,5,6,5]
rank3 = [1,6,1,1,3,2,6,6,6,6,1,5,6,1,6,5,6]
rank4 = [5,3,5,6,6,5,5,3,2,1,4,2,3,4,2,4,4]
rank5 = [4,5,4,5,5,3,4,4,3,3,3,3,2,5,4,2,1]
rank6 = [3,4,5,4,4,5,4,1,4,2,5,1,5,6,3,1,2]


guessed = np.array([7, 12, 13, 8, 11, 6])
n = sum(guessed) / 3
guessed_norm = guessed / n

avg_rank = [np.mean(rank1),
           np.mean(rank2),
           np.mean(rank3),
           np.mean(rank4),
           np.mean(rank5),
           np.mean(rank6)]

buckets = ["1 (generert)", "2", "3", "4 (generert)", "5 (generert)", "6"]

plt.barh(buckets, guessed_norm)
plt.ylabel("Dikt nr.")
plt.xlabel("Frekvens gjettet generert")
plt.show()
plt.barh(buckets, avg_rank)
plt.ylabel("Dikt nr.")
plt.xlabel("Gjennomsnittlig plassering")
plt.show()